In [19]:
# !pip install colab-ssh
!pip install -Uqq fastbook

BASE_DIR = /content/drive/MyDrive/Colab Notebooks/fastai-practice
%cd BASE_DIR

/content/drive/MyDrive/Colab Notebooks/fastai-practice


In [20]:
f = open("ngrok_key.txt", 'r')
NGROK_TOKEN = f.read()
f.close()

f = open("azure_key.txt", 'r')
AZURE_TOKEN = f.read()
f.close()

# PASSWORD = '1234'
# from colab_ssh import launch_ssh
# launch_ssh(NGROK_TOKEN, PASSWORD)

In [21]:
import fastbook
from fastai.vision.all import *
# from fastai.vision.widgets import *
# fastbook.setup_book() # to connect the colab notebook to google drive using an authentication token

In [22]:
import requests

def search_images_bing(key, term, min_sz=128, max_images=150):    
     params = {'q':term, 'count':max_images, 'min_height':min_sz, 'min_width':min_sz}
     headers = {"Ocp-Apim-Subscription-Key":AZURE_TOKEN}
     search_url = "https://api.bing.microsoft.com/v7.0/images/search"
     response = requests.get(search_url, headers=headers, params=params)
     response.raise_for_status()
     search_results = response.json()    
     return L(search_results['value'])

In [23]:
results = search_images_bing(AZURE_TOKEN, 'grizzly bear')
ims = results.attrgot('content_url')
len(ims)

150

In [24]:
bear_types = 'grizzly','black','teddy'
path = Path(f'{VA}/bears')

In [25]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(AZURE_TOKEN, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [26]:
fns = get_image_files(path)
fns

(#0) []

In [27]:
!git config --global user.email 'intaek428@gmail.com'
!git config --global user.name 'ITHwang'

!git add .
!git commit -m '비밀번호 txt로 옮김'
!git push

[master 446d0a9] 비밀번호 txt로 옮김
 2 files changed, 3 insertions(+), 1 deletion(-)
 create mode 100644 .gitignore
 rewrite fastai_chap2.ipynb (96%)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 1.23 KiB | 251.00 KiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/ITHwang/fastai-practice.git
   b6fd447..446d0a9  master -> master
